In [4]:
import numpy as np
import pandas as pd
from numpy.linalg import inv, det, eig
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms
from scipy.stats import chi2, multivariate_normal

In [2]:
def getGussianParam(X):
    """
    计算正态分布参数
    """
    u = np.mean(X, axis=0)
    cov = np.cov(X.T)
    cov_inv = inv(cov)
    cov_det = det(cov)
    return {'u': u, 'cov': cov, 'cov_inv': cov_inv, 'cov_det': cov_det}

def getGussianProb(u, cov, cov_inv, cov_det, x):
    """
    计算正态分布概率
    """
    y = np.dot(x - u, cov_inv)
    y = np.dot(y, x - u)
    y = np.exp(-0.5 * y)
    const = 1 / (((3.1415 * 2) ** (len(x) / 2)) * (cov_det ** 0.5))
    return y * const

def mapConfidenceRadius(p, df=2):
    """
    置信区间和半径的关系,
    (x-u)'S**-1(x-u)=c**2，
    服从chi-square分布，返回c**2
    """
    rv = chi2(2)
    return rv.ppf(p)

def isInsideConf(x, u, cov, c2):
    """
    计算是否在置信区间内
    """
    cov_inv = inv(cov)
    y = np.dot(x-u, cov)
    y = np.dot(y, x-u)
    re = 1 if y <= c2 else 0
    return re

def drawEllipse(s1, s2, c):
    """
    计算中心点在原点且没有变形的椭圆的坐标，
    s1; s2: 方差; c: 半径
    """
    a = s1 * c
    b = s2 * c
    x1 = np.linspace(-a, a)
    x2 = -b/a*np.sqrt(a**2 - x1**2)
    x1_second = np.flipud(x1)
    x2_second = np.flipud(-x2)
    x1 = np.concatenate((x1, x1_second))
    x2 = np.concatenate((x2, x2_second))
    return x1, x2

def drawEllipse2(u, cov, c):
    """
    画中心点为u，变化为cov，半径为c的椭圆
    """
    w, v = eig(cov)
    x1, x2 = drawEllipse(s1=np.sqrt(w[0]), s2=np.sqrt(w[1]), c=c)
    Y = np.array([x1, x2])
    X = np.dot(v, Y).T + u
    return X

def drawScatterAndConf(data, ps=[0.99]):
    """
    scatter plot & confidence interval
    """
    params = getGussianParam(data[['x1', 'x2']].values)
    plt.grid()
    plt.scatter(data['x1'].values, data['x2'].values)
    for p in ps:
        c2 = mapConfidenceRadius(p)
        X = drawEllipse2(params['u'], params['cov'], np.sqrt(c2))
        plt.plot(X[:, 0], X[:, 1])
    plt.show()

In [5]:
rv = multivariate_normal([0.5, -0.2], [[2.0, 0.3], [0.3, 0.5]])
data = np.array([rv.rvs() for i in range(100)])

array([-1.89316243, -0.51540864])